This notebook shows how to use `tfe.queue.FIFOQueue`.

# Setup

In [1]:
import tensorflow as tf
import tf_encrypted as tfe

from utils import print_in_notebook

Turn on TFE profling flags since we want to inspect with TensorBoard below.

In [2]:
%load_ext tensorboard.notebook

TENSORBOARD_DIR = "/tmp/tensorboard"

tfe.set_tfe_trace_flag(True)
tfe.set_tfe_events_flag(True)
tfe.set_log_directory(TENSORBOARD_DIR)

INFO:tf_encrypted:Tensorflow encrypted is dumping computation traces
INFO:tf_encrypted:Writing event and trace files to '/tmp/tensorboard'


('Tensorflow encrypted is monitoring statistics for each', 'session.run() call using a tag')


# Session

We here prepare a session that is ready to use and be interleaved with graph construction. Note that for this to work we have to specify all players up front.

In [3]:
tf.Session.reset('')
tf.reset_default_graph()

!rm -rf {TENSORBOARD_DIR}

players = ['server0', 'server1', 'server2', 'inputter', 'result-receiver']
tfe.set_config(tfe.LocalConfig(players))
tfe.set_protocol(tfe.protocol.Pond())

sess = tfe.Session(disable_optimizations=True)

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2', 'inputter', 'result-receiver']


We can then create and use our queue, following how this would be done in ordinary TensorFlow.

In [4]:
shape = (5, 5)

q = tfe.queue.FIFOQueue(capacity=10, shape=shape)

In [5]:
x = tfe.define_private_input('inputter', lambda: tf.fill(shape, 5))

enqueue_op = q.enqueue(x)

In [6]:
y = q.dequeue()
z = y + y

dequeue_op = tfe.define_output('result-receiver', z, print_in_notebook)

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [7]:
sess.run(enqueue_op, tag='enqueue')

print("Queue status: {}/{}".format(sess.run(q.size()), q.capacity))

Queue status: 1/10


In [8]:
sess.run(dequeue_op, tag='dequeue')

print("Queue status: {}/{}".format(sess.run(q.size()), q.capacity))

[[10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]]
Queue status: 0/10


# Inspection

In [9]:
%tensorboard --logdir {TENSORBOARD_DIR}

Reusing TensorBoard on port 6010 (pid 64858), started 22:38:10 ago. (Use '!kill 64858' to kill it.)